In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Scikit-learn nos ofrece una variedad ampliada de modelos Naive Bayes, para este problema usamos MultinomialNB que es pensado para este tipo de problemas
from sklearn.naive_bayes import MultinomialNB   

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

# TP3: Detector de SPAM

Uno de los problemas más comunes en la clasificación es la detección de correos electrónicos SPAM. Uno de los primeros modelos utilizados para abordar este problema fue el clasificador de Bayes ingenuo. La detección de SPAM es un problema persistente en el mundo digital, ya que los spammers continúan adaptando sus estrategias para eludir los filtros de correo no deseado. Además del clasificador de Bayes ingenuo, se han desarrollado y utilizado una variedad de técnicas más avanzadas en la detección de SPAM, que incluyen algoritmos de aprendizaje automático, redes neuronales y métodos basados en reglas.

En este trabajo práctico, utilizaremos un conjunto de datos que consta de 4601 observaciones de correos electrónicos, de los cuales 2788 son correos legítimos y 1813 son correos SPAM. Dado que el contenido de los correos electrónicos es un tipo de dato no estructurado, es necesario procesarlo de alguna manera. Para este conjunto de datos, ya se ha aplicado un procesamiento típico en el Procesamiento del Lenguaje Natural (NLP), que consiste en contar la frecuencia de palabras observadas en los correos.

El procesamiento de lenguaje natural (NLP) desempeña un papel fundamental en la detección de SPAM, ya que permite analizar el contenido de los correos electrónicos y extraer características relevantes para la clasificación. Además de contar la frecuencia de palabras, se pueden utilizar técnicas más sofisticadas, como la extracción de características semánticas y el análisis de sentimientos, para mejorar la precisión de los modelos de detección de SPAM.

En este proceso, se cuenta la cantidad de ocurrencias de cada palabra en los diferentes correos.

![spam counter](./spam.png)

Con el fin de preservar la privacidad de los mensajes, la frecuencia de palabras se encuentra normalizada. El conjunto de datos está compuesto por 54 columnas de atributos que se denominan:

- `word_freq_XXXX`: Donde `XXXX` es la palabra o símbolo. Los valores son enteros que van de 0 a 20k.

Además, hay una columna adicional llamada `spam`, que es 1 si el correo es SPAM o 0 si no lo es.

Los clasificadores de Bayes ingenuos fueron los primeros filtros utilizados por las aplicaciones de correo electrónico, basados en este principio de palabras. La idea es que, partiendo de un dato a priori sobre la probabilidad de que un correo sea SPAM o no, ciertas palabras nos indicarán que la probabilidad a posteriori, dadas esas palabras, es más probable que el correo sea SPAM o no.

In [9]:
dataset = pd.read_csv("dataset/spambase.csv") # cargando los datos desde un CSV
dataset.head(100)

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,spam
0,0,640,640,0,320,0,0,0,0,0,...,0,0,0,0,0,0,778,0,0,1
1,210,280,500,0,140,280,210,70,0,940,...,0,0,0,0,132,0,372,180,48,1
2,60,0,710,0,1230,190,190,120,640,250,...,60,0,0,10,143,0,276,184,10,1
3,0,0,0,0,630,0,310,630,310,630,...,0,0,0,0,137,0,137,0,0,1
4,0,0,0,0,630,0,310,630,310,630,...,0,0,0,0,135,0,135,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,460,0,0,0,460,0,0,0,...,0,0,0,0,0,0,0,0,0,1
96,0,350,700,0,350,0,0,0,0,0,...,0,0,0,0,122,0,1284,0,0,1
97,0,430,430,0,430,0,0,0,0,0,...,0,0,0,0,147,0,736,0,0,1
98,0,0,0,0,0,600,0,0,0,1210,...,0,0,0,207,518,414,310,0,0,1


Para obtener las palábras más usadas podemos hacer un `groupby`:

In [35]:
column_sum = dataset.groupby(by="spam", as_index=False).sum()
column_sum

,spam,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,...,word_freq_re,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#
0,0,204860,681569,559220,2470,504739,124190,26160,107100,106080,...,1159138,800669,22840,142820,140182,442116,63242,306634,32475,60536
1,1,276190,298510,732080,298550,931799,317050,499309,377360,308320,...,226790,26700,2210,3810,37299,197563,14864,931352,316328,143003


Y despues se pueden combinar las columnas en usando [pd.melt](https://pandas.pydata.org/docs/reference/api/pandas.melt.html)

In [14]:
# Obtenemos los atributos y target
X = (dataset.drop(columns="spam") * 100).astype(int)
#X = dataset2.drop(columns="spam")
y = dataset["spam"]

Se separa el dataset en entrenamiento y evaluación

In [17]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.3)

Escalamos para aplicar en regresión logística

In [20]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Lo transformamos en DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

In [23]:
X_train_scaled

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_re,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#
0,0.025381,0.014006,0.198039,0.0,0.080,0.224090,0.068776,0.000000,0.152091,0.009001,...,0.005,0.004535,0.00000,0.000,0.000000,0.011280,0.000000,0.015087,0.026320,0.000756
1,0.000000,0.005602,0.015686,0.0,0.000,0.022409,0.000000,0.026403,0.000000,0.000000,...,0.000,0.000000,0.00000,0.008,0.005017,0.011382,0.000000,0.000000,0.009162,0.000000
2,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.214221,...,0.000,0.000000,0.00000,0.000,0.000000,0.000000,0.000000,0.013609,0.000000,0.000000
3,0.015228,0.000000,0.011765,0.0,0.019,0.000000,0.000000,0.000000,0.138783,0.000000,...,0.000,0.000000,0.02765,0.000,0.082782,0.014664,0.020526,0.000000,0.003165,0.000000
4,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.178218,0.000000,0.000000,...,0.000,0.000000,0.00000,0.054,0.000000,0.017022,0.000000,0.002556,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,0.000000,0.022409,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.00000,0.000,0.000000,0.007691,0.000000,0.000000,0.000000,0.000000
3216,0.000000,0.000000,0.017647,0.0,0.000,0.000000,0.000000,0.014851,0.000000,0.000000,...,0.000,0.000000,0.00000,0.000,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000
3217,0.043147,0.000000,0.015686,0.0,0.042,0.022409,0.011004,0.069307,0.015209,0.007201,...,0.000,0.000000,0.00000,0.000,0.000000,0.008306,0.009723,0.002925,0.002166,0.000000
3218,0.000000,0.000000,0.000000,0.0,0.000,0.193277,0.000000,0.000000,0.043726,0.000000,...,0.000,0.031293,0.00000,0.000,0.007526,0.000000,0.011883,0.000000,0.000000,0.000000
